In [65]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [67]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import scipy

In [69]:
from synthetic_data import stationary_pb_ts, nonstationary_ts_n
from HOI.preprocessings import compute_kernel, compute_kernel_n
from HOI.tests import test_independence, dhsic_permutation, test_lancaster, estimate_tail_head

In [70]:
import matplotlib.pyplot as plt

In [13]:
rejects = 0
for i in tqdm(np.arange(100)):
    df = stationary_pb_ts(t_time = 100, d = 0.8, mode = "case1", a=0.5)
    data_dict, kernel_dict = compute_kernel(df)
    _, _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']],
                             [data_dict['d1'], data_dict['d2'], data_dict['d3']],
                              n_perms=1000, alpha=0.05, mode = 'shifting')
    rejects = rejects + reject
print(rejects)

  0%|          | 0/100 [00:00<?, ?it/s]

31
